# **시계열 데이터 처리**


## **1.환경준비**

### **(1) 라이브러리 불러오기**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### **(2) 데이터 준비**

* 원본데이터 불러오기

In [31]:
sales = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_sales_simple.csv")
products = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_product_master.csv")

* 다룰 데이터 만들기
    * 아래 코드를 이해하지 못해도 상관 없습니다.
    * 그냥 실행해주세요.

In [32]:
# 판매액 계산하기
temp = pd.merge(sales, products)
temp['Amt' ] = temp['Qty'] * temp['Price']
temp['Amt'] = (temp['Amt']/1000).round()  # 단위 1000달러

# 집계
data1 = temp.groupby(['Date', 'Category'], as_index = False)['Amt'].sum()
data2 = temp.groupby(['Date'], as_index = False)['Amt'].sum()

# 피봇
data11 = data1.pivot(index = 'Date', columns = 'Category', values = 'Amt').reset_index()

# 합치기
data = pd.merge(data2, data11)

* 우리가 다룰 데이터

In [33]:
data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods
0,2013-01-01,20.0,7.0,4.0,6.0,3.0
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0


## **2.날짜 요소 추출**

### **(1) 날짜 타입으로 변환**
* pd.to_datetime(날짜데이터, format = '입력되는 날짜 형식')
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             31 non-null     object 
 1   Amt              31 non-null     float64
 2   Drink            31 non-null     float64
 3   Food             31 non-null     float64
 4   Grocery          31 non-null     float64
 5   Household Goods  31 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.6+ KB


In [35]:
data['Date'] = pd.to_datetime(data['Date'])

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             31 non-null     datetime64[ns]
 1   Amt              31 non-null     float64       
 2   Drink            31 non-null     float64       
 3   Food             31 non-null     float64       
 4   Grocery          31 non-null     float64       
 5   Household Goods  31 non-null     float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 1.6 KB


* format = ''
    * pd.to_datetime(date, format = '%d/%m/%Y') # format = '%d/%m/%Y' 입력되는 날짜가 이런 형태야~~ 라고 알려주는 옵션
    * https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [37]:
# pd.to_datetime(date, format = '%d/%m/%Y') # format = '%d/%m/%Y' 입력되는 날짜가 이런 형태야~~ 라고 알려주는 옵션
date = pd.Series(['03-01-2023', '03-02-2023', '03-03-2023'])
date = pd.to_datetime(date)
date

,0
0,2023-03-01
1,2023-03-02
2,2023-03-03


In [38]:
# 만약 dd-mm-yyyy 이 맞다면!
date = pd.Series(['03-01-2023', '03-02-2023', '03-03-2023'])
date = pd.to_datetime(date, format = '%d-%m-%Y') #입력받은 날짜 데이터 형식이 '%d-%m-%Y'!
date

,0
0,2023-01-03
1,2023-02-03
2,2023-03-03


### **(2) 날짜 요소 추출**

In [10]:
# 연도
date.dt.year

,0
0,2023
1,2023
2,2023


In [11]:
# 월
date.dt.month

,0
0,1
1,2
2,3


In [12]:
# 일
date.dt.day

,0
0,3
1,3
2,3


In [13]:
# 요일
date.dt.weekday

,0
0,1
1,4
2,4


In [14]:
# 요일 이름
date.dt.day_name()

,0
0,Tuesday
1,Friday
2,Friday


<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] data의 Date는 이미 날짜 타입으로 변환되어 있습니다.  
* 다음의 항목을 열로 추가하시오.
    * 요일(이름)
    * 주차

In [15]:
data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods
0,2013-01-01,20.0,7.0,4.0,6.0,3.0
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0


In [39]:
data['Date'] = pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             31 non-null     datetime64[ns]
 1   Amt              31 non-null     float64       
 2   Drink            31 non-null     float64       
 3   Food             31 non-null     float64       
 4   Grocery          31 non-null     float64       
 5   Household Goods  31 non-null     float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 1.6 KB


In [40]:
data['WeekDay'] = data['Date'].dt.day_name()
data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods,WeekDay
0,2013-01-01,20.0,7.0,4.0,6.0,3.0,Tuesday
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0,Wednesday
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0,Thursday
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0,Friday
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0,Saturday


In [41]:
data['Week'] = data['Date'].dt.isocalendar().week
data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods,WeekDay,Week
0,2013-01-01,20.0,7.0,4.0,6.0,3.0,Tuesday,1
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0,Wednesday,1
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0,Thursday,1
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0,Friday,1
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0,Saturday,1


## **3.시간에 따른 흐름 추가하기**

### **(1) shift**
* 시계열 데이터에서 시간의 흐름 전후로 정보를 이동시킬 때 사용
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html

In [21]:
temp = data.loc[:,['Date','Amt']]

In [22]:
# 전날 매출액 열을 추가합시다.
temp['Amt_lag'] = temp['Amt'].shift() # default = 1

# 전전날 매출액 열을 추가.
temp['Amt_lag2'] = temp['Amt'].shift(2) # 2행 shift

# 다음날 매출액 열을 추가합시다.
temp['Amt_lag_1'] = temp['Amt'].shift(-1)

temp.head()

,Date,Amt,Amt_lag,Amt_lag2,Amt_lag_1
0,2013-01-01,20.0,NaN,NaN,3938.0
1,2013-01-02,3938.0,20.0,NaN,2885.0
2,2013-01-03,2885.0,3938.0,20.0,2907.0
3,2013-01-04,2907.0,2885.0,3938.0,3831.0
4,2013-01-05,3831.0,2907.0,2885.0,4066.0


shift에 대한 이해를 위한 예시

In [23]:
# 먼저, temp 데이터프레임 생성 (Date, Amt 라는 열 존재)
import pandas as pd

data = {'Date': pd.date_range(start='2023-01-01', periods=5, freq='D'),
        'Amt': [100, 150, 200, 250, 300]}
temp = pd.DataFrame(data)
print(temp)

        Date  Amt
0 2023-01-01  100
1 2023-01-02  150
2 2023-01-03  200
3 2023-01-04  250
4 2023-01-05  300


날짜가 아닌 행의 위치를 기준으로 이동시키는데, 이때 행의 위치는 0부터 시작하는 인덱스를 기준으로 데이터를 위나 아래로 이동시킵니다.

In [25]:
# 전날 매출액 열 추가 (shift() 기본값은 1)
# 이렇게 하면 Amt 열의 값을 한 행 아래로 이동한다. 첫 번째 행에는 이전 값이 없으므로 NaN으로 채워짐.

temp['Amt_lag'] = temp['Amt'].shift()  # 기본값 periods=1
print(temp)

        Date  Amt  Amt_lag
0 2023-01-01  100      NaN
1 2023-01-02  150    100.0
2 2023-01-03  200    150.0
3 2023-01-04  250    200.0
4 2023-01-05  300    250.0


In [26]:
# 전전날 매출액 열 추가 (shift(2))
# Amt 열의 값을 두 행 아래로 이동한다. 첫 번재, 두 번째 행에는 두 행 전의 값이 없으므로 NaN 채워짐.

temp['Amt_lag2'] = temp['Amt'].shift(2)  # periods=2
print(temp)

        Date  Amt  Amt_lag  Amt_lag2
0 2023-01-01  100      NaN       NaN
1 2023-01-02  150    100.0       NaN
2 2023-01-03  200    150.0     100.0
3 2023-01-04  250    200.0     150.0
4 2023-01-05  300    250.0     200.0


In [27]:
# 다음날 매출액 열 추가 (shift(-1))
# Amt 열의 값을 한 행 위로 이동. 마지막 행에는 다음 값이 없으므로 NaN이 채워짐.

temp['Amt_lag_1'] = temp['Amt'].shift(-1)  # 1행 위로 이동 (다음 날 값)
print(temp)

        Date  Amt  Amt_lag  Amt_lag2  Amt_lag_1
0 2023-01-01  100      NaN       NaN      150.0
1 2023-01-02  150    100.0       NaN      200.0
2 2023-01-03  200    150.0     100.0      250.0
3 2023-01-04  250    200.0     150.0      300.0
4 2023-01-05  300    250.0     200.0        NaN


그리고 기준을 잡는 건 temp['Amt']라 설정을 했으므로 이걸 기준으로 이동시킵니다.

### **(2) rolling + 집계함수**
* 시간의 흐름에 따라 일정 기간 동안 평균을 이동하면서 구하기
* .rolling : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
* rolling(n) :
    * n 기본값은 1
    * min_periods : 최소 데이터수

In [28]:
# 7일 이동평균 매출액을 구해 봅시다.
temp['Amt_MA7_1'] = temp['Amt'].rolling(7).mean()
temp['Amt_MA7_2'] = temp['Amt'].rolling(7, min_periods = 1).mean()
temp.head(10)

,Date,Amt,Amt_lag,Amt_lag2,Amt_lag_1,Amt_MA7_1,Amt_MA7_2
0,2023-01-01,100,NaN,NaN,150.0,NaN,100.0
1,2023-01-02,150,100.0,NaN,200.0,NaN,125.0
2,2023-01-03,200,150.0,100.0,250.0,NaN,150.0
3,2023-01-04,250,200.0,150.0,300.0,NaN,175.0
4,2023-01-05,300,250.0,200.0,NaN,NaN,200.0


(1) Amt 열의 값을 연속된 7개의 행(7일)을 기준으로 이동평균을 계산합니다. 첫 번째 행부터 여섯 번째 행까지는 7개의 값이 부족하기 때문에 평균을 계산할 수 없어서 NaN이 됩니다.
일곱 번째 행부터는 이전 7개의 값의 평균을 계산합니다.

(2) 7개의 행(7일)을 기준으로 이동평균을 계산하되, 최소 1개의 값이 있으면 평균을 계산합니다.최소 1개의 값만 있으면 평균을 계산합니다. 즉, 7개의 값이 모두 채워지지 않아도 그 시점까지의 평균을 계산합니다. 첫 번째 행에서는 1개의 값, 두 번째 행에서는 2개의 값, ..., 일곱 번째 행부터는 7개의 값을 기준으로 평균을 계산합니다.

### (3) **diff**
* 특정 시점 데이터, 이전시점 데이터와의 차이 구하기
* .diff : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html

In [29]:
# 7일 이동평균 매출액을 구해 봅시다.
temp['Amt_D1'] = temp['Amt'].diff()
temp['Amt_D2'] = temp['Amt'].diff(2)
temp.head(10)

,Date,Amt,Amt_lag,Amt_lag2,Amt_lag_1,Amt_MA7_1,Amt_MA7_2,Amt_D1,Amt_D2
0,2023-01-01,100,NaN,NaN,150.0,NaN,100.0,NaN,NaN
1,2023-01-02,150,100.0,NaN,200.0,NaN,125.0,50.0,NaN
2,2023-01-03,200,150.0,100.0,250.0,NaN,150.0,50.0,100.0
3,2023-01-04,250,200.0,150.0,300.0,NaN,175.0,50.0,100.0
4,2023-01-05,300,250.0,200.0,NaN,NaN,200.0,50.0,100.0


<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] data의 Grocery가 매출이 가장 높습니다. 이에 대해서 다음의 열을 추가해 봅시다.

* 전날 매출액
* 7일 전(전주 동 요일) 매출액
* 3일 이동평균 매출액
* 전날대비 매출액 증감여부 (증가 1, 감소 -1, 동일 0)


In [42]:
data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods,WeekDay,Week
0,2013-01-01,20.0,7.0,4.0,6.0,3.0,Tuesday,1
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0,Wednesday,1
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0,Thursday,1
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0,Friday,1
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0,Saturday,1


In [45]:
# 전날 매출액
data['Grocery_lag1'] = data['Grocery'].shift()

# 7일 전(전주 동 요일) 매출액
data['Grocery_lag7'] = data['Grocery'].shift(7)

# 3일 이동평균 매출액
data['Grocery_MA3'] = data['Grocery'].rolling(3, center=True, min_periods=1).mean()

# 전날대비 매출액 증감여부 (증가 1, 감소 -1, 동일 0)
# data['Grocery_diff'] = np.where(data['Grocery'].diff() > 0, 1, -1)
# data['Grocery_diff'] = np.where(data['Grocery_diff'] == 0, 0, data['Grocery_diff'])

data['Grocery_diff'] = np.where(data['Grocery'].diff(1) > 0, 1,
                                np.where(data['Grocery'].diff(1) < 0, -1, 0))

data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods,WeekDay,Week,Grocery_lag1,Grocery_lag7,Grocery_MA3,Grocery_diff
0,2013-01-01,20.0,7.0,4.0,6.0,3.0,Tuesday,1,NaN,NaN,834.500000,0
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0,Wednesday,1,6.0,NaN,963.666667,1
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0,Thursday,1,1663.0,NaN,1379.000000,-1
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0,Friday,1,1222.0,NaN,1344.666667,1
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0,Saturday,1,1252.0,NaN,1508.333333,1
